In [3]:
pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.7 MB/s eta 0:00:00


In [17]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, Bidirectional
import gensim.downloader as api

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Loading CSV**

In [2]:
text = pd.read_csv('/content/spam.csv', sep=',', encoding='latin-1')
text.drop(columns=text.columns[text.columns.str.contains('unnamed', case=False)], inplace=True)
print(text.head(10))

     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
5  spam  FreeMsg Hey there darling it's been 3 week's n...
6   ham  Even my brother is not like to speak with me. ...
7   ham  As per your request 'Melle Melle (Oru Minnamin...
8  spam  WINNER!! As a valued network customer you have...
9  spam  Had your mobile 11 months or more? U R entitle...


**Pre processing**

In [5]:
def preprocessing(text):
  #Removing Contractions
  text = contractions.fix(text)

  # Removing HTML tags
  text_cleaned = re.sub(r'<[^>]+>', '', text)

  # Removing URLs
  text_cleaned = re.sub(r'http\S+|www\S+|https\S+', '', text_cleaned)

  # Converting to lowercase and removing non-alphanumeric characters
  text_cleaned = re.sub(r'[^a-zA-Z\s]', '', text_cleaned.lower())

  # Tokenization
  tokens = word_tokenize(text_cleaned)

  # Removing stopwords
  stop_words = set(stopwords.words('english'))
  tokens_without_sw = [word for word in tokens if word not in stop_words]

  # Regex: Remove punctuation marks
  tokens_without_punctuation = [word for word in tokens_without_sw if re.match(r'^\w+$', word)]

  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens_without_punctuation]

  return " ".join(lemmatized_tokens)

text['processed_text'] = text['v2'].apply(preprocessing)
print(text)

        v1                                                 v2  \
0      ham  Go until jurong point, crazy.. Available only ...   
1      ham                      Ok lar... Joking wif u oni...   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3      ham  U dun say so early hor... U c already then say...   
4      ham  Nah I don't think he goes to usf, he lives aro...   
...    ...                                                ...   
5567  spam  This is the 2nd time we have tried 2 contact u...   
5568   ham              Will Ì_ b going to esplanade fr home?   
5569   ham  Pity, * was in mood for that. So...any other s...   
5570   ham  The guy did some bitching but I acted like i'd...   
5571   ham                         Rofl. Its true to its name   

                                         processed_text  
0     go jurong point crazy available bugis n great ...  
1                                 ok lar joking wif oni  
2     free entry wkly comp win fa cup final t

**Making X and Y, X=Preprocessed text and Y=Labels**

In [8]:
X = text['processed_text'].values
y = np.where(text['v1'] == 'spam', 1, 0)

**Text to sequence, Padding Seq, SplitData, Word2Vec**

In [9]:
# Text to Sequence Conversion
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

# Padding Sequences
max_sequence_length = 100
sequences_padded = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Split Data
X_train, X_test, y_train, y_test = train_test_split(sequences_padded, y, test_size=0.2, random_state=42)

# Load pre-trained Word2Vec model (glove-wiki-gigaword-100)
word2vec_model = api.load("glove-wiki-gigaword-100")

# Create the embedding matrix from the Word2Vec model
embedding_dim = 100
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, idx in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[idx] = word2vec_model[word]

[==================================================] 100.0% 128.1/128.1MB downloaded


**Unidirectional RNN Model**

In [15]:
# Build Unidirectional RNN Model with ReLU activation
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/5
140/140 [==============================] - 7s 28ms/step - loss: 0.4274 - accuracy: 0.8661 - val_loss: 0.3949 - val_accuracy: 0.8655
Epoch 2/5
140/140 [==============================] - 4s 27ms/step - loss: 0.4035 - accuracy: 0.8661 - val_loss: 0.6621 - val_accuracy: 0.8655
Epoch 3/5
140/140 [==============================] - 5s 37ms/step - loss: 0.4631 - accuracy: 0.8661 - val_loss: 0.3918 - val_accuracy: 0.8655
Epoch 4/5
140/140 [==============================] - 4s 26ms/step - loss: 0.3732 - accuracy: 0.8661 - val_loss: 0.5040 - val_accuracy: 0.8655
Epoch 5/5
35/35 [==============================] - 0s 8ms/step - loss: 0.3481 - accuracy: 0.8655
Test Loss: 0.348071426153183
Test Accuracy: 0.865470826625824


**Bidirectional RNN**

In [18]:
# Build Bidirectional RNN Model with ReLU activation
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(SimpleRNN(64, activation='relu')))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/5
140/140 [==============================] - 10s 58ms/step - loss: 0.3104 - accuracy: 0.8847 - val_loss: 0.1593 - val_accuracy: 0.9426
Epoch 2/5
140/140 [==============================] - 6s 44ms/step - loss: 0.1382 - accuracy: 0.9585 - val_loss: 0.1231 - val_accuracy: 0.9507
Epoch 3/5
140/140 [==============================] - 8s 55ms/step - loss: 0.0922 - accuracy: 0.9684 - val_loss: 0.1018 - val_accuracy: 0.9623
Epoch 4/5
140/140 [==============================] - 6s 42ms/step - loss: 0.0725 - accuracy: 0.9769 - val_loss: 0.1004 - val_accuracy: 0.9650
Epoch 5/5
35/35 [==============================] - 0s 14ms/step - loss: 0.0996 - accuracy: 0.9659
Test Loss: 0.09957161545753479
Test Accuracy: 0.9659192562103271


**LSTM**

In [19]:
# Build Bidirectional LSTM Model with ReLU activation
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, activation='relu')))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/5
140/140 [==============================] - 21s 123ms/step - loss: 42.5086 - accuracy: 0.8914 - val_loss: 0.2561 - val_accuracy: 0.9085
Epoch 2/5
140/140 [==============================] - 16s 116ms/step - loss: 1.5553 - accuracy: 0.9096 - val_loss: 0.3973 - val_accuracy: 0.8682
Epoch 3/5
140/140 [==============================] - 17s 119ms/step - loss: 0.3265 - accuracy: 0.8735 - val_loss: 0.4396 - val_accuracy: 0.8735
Epoch 4/5
140/140 [==============================] - 18s 128ms/step - loss: 0.2496 - accuracy: 0.9161 - val_loss: 0.2901 - val_accuracy: 0.9013
Epoch 5/5
35/35 [==============================] - 1s 30ms/step - loss: 0.2359 - accuracy: 0.9426
Test Loss: 0.2359250783920288
Test Accuracy: 0.9426009058952332
